The functions used to create the fire_climate_data function

In [3]:
import requests
import pandas as pd
import io
from shapely import Point, Polygon
import re

#Fire regions established the california fire science consortium 

region_a = Polygon([(38.874909265688,-121.53076171875),(34.760701047279,-118.36669921875),(36.263007941398,-116.30126953125),(38.874909265688,-120.08056640625)])
region_b = Polygon([(32.436904704681,-117.67456054688),(32.733128299191,-114.11499023438),(34.471596825755,-114.20288085938),(36.174592339967,-116.22436523438),(34.74287008807,-118.28979492188),(34.23577762138,-117.14721679688),(32.91777007614,-119.56420898438)])
region_c = Polygon([(41.959490098925,-124.4091796875),(41.959490098925,-119.8388671875),(39.087436032159,-119.9267578125),(39.053318106741,-124.80468755)])
region_d = Polygon([(39.007575869284,-123.804931640625),(32.931610922518,-119.5861816406),(34.267569211232,-117.36694335938),(34.720305876154,-118.44360351562),(39.007575869284,-121.76147460938)])
regions = [region_a,region_b, region_c, region_d]

#takes the fire location and returns region of fire 
def find_region(fire_location):
  for region in regions:
    if region.contains(fire_location.centroid):
      return region
  raise RuntimeError ('fire not in regions, expand regions')




#takes the region and returns a list of stations in region and their coordinates
def region_station_list(region): 
  
  file = open('./ccsl.txt', 'r')
  count = 0
  station_list = []
  while True:
    count += 1
    line = file.readline()

    if not line:
      break
    
    #convert the station into a shapely point to check for containment 
    station_coordinate_lat = line[84:93]
    station_coordinate_lon = line[95:105] #x99*99'99"
    if ('*' not in station_coordinate_lat) and ('*' not in station_coordinate_lon):
      continue
    station_coordinate_lat_D = station_coordinate_lat.split('*')[0] # 
    station_coordinate_lat_M = station_coordinate_lat.split('*')[1].split('\'')[0]
    station_coordinate_lat_S = station_coordinate_lat.split('*')[1].split('\'')[1]
    station_coordinate_lon_D = station_coordinate_lon.split('*')[0]
    station_coordinate_lon_M = station_coordinate_lon.split('*')[1].split('\'')[0]
    station_coordinate_lon_S = station_coordinate_lon.split('*')[1].split('\'')[1]

    station_coordinate_lat = station_coordinate_lat_D + station_coordinate_lat_M.div(60) + station_coordinate_lat_S.div(3600)
    station_coordinate_lon = station_coordinate_lon_D + station_coordinate_lon_M.div(60) + station_coordinate_lon_S.div(3600)
    station_coordinate = (station_coordinate_lat, station_coordinate_lon)
    station_coordinate = Point(station_coordinate)
    
    #check for containment inside the polygon to create a string of stations 
    if region.contains(station_coordinate):  
      station_id = line[75:83]
      station_list.append((station_id, station_coordinate))
  file.close()
  return station_list 
  



#takes the list of stations and fire location to return stations within a degree of distance from fire
def closest_to_fire_center(station_list, fire_location):
  fire_center = fire_location.centroid
  request_station_list = []
  for station in station_list:
    if station[1].distance(fire_center) < 1:
      request_station_list.append(station[0])

  region_station_list = ','.join(request_station_list) #string to be used in api reguest 
  return region_station_list



#takes a string of closest stations and requests their data 
def request_station_data(closest_stations):
  #polygon to a string for api use 
  header = {'token': 'SXCObXMuhoovZxrVZpurQqmsHdvJWyUJ'}
  r=requests.get(f'https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-summary-of-the-year&stations={closest_stations}&&startDate=1950-01-01T00:00:00z&&endDate=2022-01-01&format=csv&bbox=-124.40959,32.534156,-114.131211,42.009518', headers=header)
  if r.status_code != 200:
    print(r.status_code)
    print(r.text)
  df = pd.read_csv(io.StringIO(r.text))
  return df


#takes the station data returns a dataframe row of the averaged values for year of fire 
def filter_station_data(station_df, fire_year):
  station_df = station_df[station_df['YEAR_'] == fire_year]
  station_df = station_df.mean()

fire_climate_data takes the fire's location and year, then returns the average of the climate data from noaa stations close to the fire 

In [ ]:
def fire_climate_data(fire_location, fire_year): #function that returns the average local climate data for year of the fire as a row to add to dataset 
  region = find_region(fire_location)
  stations = region_station_list(region)
  closest_stations = closest_to_fire_center(stations, fire_location)
  stations_df = request_station_data(closest_stations)
  filtered_df = filter_station_data(stations_df,fire_year)
  return filtered_df
  

Testing my functions 

In [ ]:
test_location =  Polygon([(38.874909265688,-121.53076171875),(34.760701047279,-118.36669921875),(36.263007941398,-116.30126953125),(38.874909265688,-120.08056640625)])